## Imports and Config

In [1]:
import os
import time
from dotenv import load_dotenv

import json
import random
from uuid import uuid4

import requests
from ast import literal_eval

import google.generativeai as genai
from pytrends.request import TrendReq

In [101]:
load_dotenv()

GEMINI_API_KEY = os.environ['GEMINI_API_KEY']
TTS_API_KEY = os.environ['TTS_API_KEY']

## Trending Topics

In [5]:
pytrends = TrendReq(hl='en-US', tz=360)

In [45]:
country = 'worldwide'
result = pytrends.trending_searches()

ix = random.randint(0, len(result))
topic = result.loc[ix].values[0]
topic

'ASU football'

## Text Prompt

In [85]:
prompt = """
You are a social media marketing expert working for a global journalistic firm, tasked with increasing engagement and viewership on Instagram, YouTube, and TikTok. Your goal is to create visually compelling, shareable content, including images, memes, and short videos.

Based on the provided trending topic "{topic}" and region "{country}", generate creative content ideas for social media posts that can go viral. Each idea should be relevant to the local culture and trends in the specified region. Follow these guidelines when generating the ideas:

1. **Categories**: The response should include three main categories:
   - **Video**: A list of 5 video clips, each with a detailed description, text overlay, and voiceover.
   - **Image**: A single image with a description, text overlay, and voiceover.
   - **Meme**: A single meme with a description, text overlay, and voiceover.

2. **Video Content Details**:
   - Provide a list of 5 elements for the video category.
   - Each element should include:
     - **Description**: What the video clip should depict, including visual elements, animations, or transitions.
     - **Text Overlay**: Text that appears on the screen to reinforce the message.
     - **Voiceover**: A short voiceover script that aligns with the visuals and conveys the key message.

3. **Image Content Details**:
   - Describe the content of the image, including visual elements and any regional cultural references.
   - Include a **Text Overlay** that adds context or reinforces the message.
   - Provide a **Voiceover** script that complements the image content.

4. **Meme Content Details**:
   - Describe the meme’s visual elements and the humorous or viral aspect.
   - Include a **Text Overlay** that adds to the meme's humor or message.
   - Provide a **Voiceover** script that adds an extra layer of humor or commentary.

5. **Song**: For all three categories (video, image, and meme), provide a **Song** section containing:
   - **Name**: The name of a trending song relevant to the topic and region.
   - **Artist**: The artist's name. Ensure the song is popular and culturally relevant.

6. **Engagement Strategy**: Incorporate features to encourage engagement, such as hashtags, call-to-actions (e.g., 'tag a friend', 'share your opinion'), or challenges (e.g., a trending dance or meme challenge).

7. **Format for Virality**: Ensure the content design maximizes shares and engagement, utilizing popular social media formats like challenges, polls, reactions, or humor. Avoid content that could evoke negative emotions or controversy.

8. **JSON Formatting Instructions**:
   - Ensure the output is formatted as valid JSON.
   - **Escape any quotes within values** using a backslash (e.g., \\"quote\\").
   - Any nested quotes inside strings should be properly escaped to ensure the JSON is valid and parsable.

Return the response as a properly formatted JSON object containing:
- **video**: A list of 5 dictionaries, where each dictionary contains keys:
  - **description**: Description of the video clip.
  - **text_overlay**: Text that appears on the video.
  - **voiceover**: Voiceover script for the clip.
- **image**: A dictionary containing keys:
  - **description**: Description of the image.
  - **text_overlay**: Text that appears on the image.
  - **voiceover**: Voiceover script for the image.
- **meme**: A dictionary containing keys:
  - **description**: Description of the meme.
  - **text_overlay**: Text that appears on the meme.
  - **voiceover**: Voiceover script for the meme.
- **song**: A dictionary containing:
  - **name**: The song name.
  - **artist**: The artist's name.
Do not include any additional text.
"""


In [86]:
genai.configure(api_key=GEMINI_API_KEY)
model = genai.GenerativeModel('gemini-1.5-flash')

In [88]:
llm_prompt = prompt.format(topic=topic, country=country)
json_text = model.generate_content(llm_prompt).text.lstrip('```json').strip('```')
response = literal_eval(json_text)

In [89]:
response

{'video': [{'description': 'A montage of ASU football players making incredible catches, tackles, and touchdowns, set to a fast-paced, energetic beat. Use slow-motion replays and dramatic close-ups for maximum impact.',
   'text_overlay': 'ASU Football:  The Future is Now! #ForksUp #SunDevilPride',
   'voiceover': "Get ready to witness the power and precision of Arizona State Football! From electrifying touchdowns to bone-crushing tackles, these Sun Devils are unstoppable.  Don't miss a moment of the action!"},
  {'description': "A behind-the-scenes look at the ASU football team's pre-game rituals and locker room energy, showcasing the camaraderie and passion of the players.",
   'text_overlay': 'What makes ASU Football so special?  Find out in our exclusive locker room footage.',
   'voiceover': "Step inside the heart of Arizona State Football! Witness the team's unwavering spirit, their intense focus, and the camaraderie that fuels their victory.  This is the essence of Sun Devil Pri

In [94]:
video_details = response['video']
video_images = [media['description'] for media in video_details]
video_texts = [media['text_overlay'] for media in video_details]
video_audios = [media['voiceover'] for media in video_details]

## Image Generation

In [51]:
POLLINATION_URI = 'https://pollinations.ai/p/{prompt}?width=1080&height=1920&seed=42&model=flux'

In [95]:
video_images

['A montage of ASU football players making incredible catches, tackles, and touchdowns, set to a fast-paced, energetic beat. Use slow-motion replays and dramatic close-ups for maximum impact.',
 "A behind-the-scenes look at the ASU football team's pre-game rituals and locker room energy, showcasing the camaraderie and passion of the players.",
 'A light-hearted, humorous video featuring ASU players reacting to funny memes and viral videos about football, showcasing their personalities off the field.',
 'A fan-submitted video of a unique or creative ASU football-themed dance or cheer, encouraging viewers to participate by sharing their own videos.',
 'A timelapse video showcasing the transformation of a stadium from an empty field to a roaring sea of fans on game day, with a focus on the excitement and energy of the crowd.']

In [96]:
img_ids = []
for img in video_images:
    res_img = requests.get(POLLINATION_URI.format(prompt=img))
    img_name = uuid4()
    img_ids.append(img_name)
    print(img_name)
    with open(f'{img_name}.jpg', 'wb') as file:
        file.write(res_img.content)
    time.sleep(2)

028c6567-5763-4fbf-aedc-c7afacc0cacc
258e7367-998a-4b61-acef-a85dbb1daecd
ba4a569a-7049-4feb-a3aa-906eee1999cd
b480bee3-0836-47df-bbd0-57e1a3adccc0
884af224-c193-467b-b248-95c9198f2437


## Audio Generation

In [110]:
!pip install gTTS


[notice] A new release of pip available: 22.3.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [111]:
from gtts import gTTS
tts = gTTS('hello')
tts.save('hello.mp3')

In [108]:
from pyt2s.services import stream_elements

# Default Voice
data = stream_elements.requestTTS('Lorem Ipsum is simply dummy text.')

# Custom Voice
# data = stream_elements.requestTTS('Lorem Ipsum is simply dummy text.', stream_elements.Voice.Russell.value)


In [109]:
with open('output.mp3', '+wb') as file:
    file.write(data)